In [ ]:
"""
Reads ModEM's Jacobian, does fancy things.

@author: vr july 2020

Created on Tue Jul  7 16:59:01 2020

@author: vrath

"""

Import required modules

In [ ]:
import os
import sys
from sys import exit as error
# import struct
import time

In [ ]:
import numpy as np
import math  as ma
import netCDF4 as nc

In [ ]:
from scipy.ndimage import \
    gaussian_filter, laplace, convolve, gaussian_gradient_magnitude,median_filter
from scipy.linalg  import norm
from sys import exit as error

In [ ]:
from modules.modem import *
from modules.util import *
#import readJac, writeJacNC, readDat, writeDatNC, sparsifyJac, readMod, rsvd

In [ ]:
import PVGeo

In [ ]:
rhoair = 1.e+17

In [ ]:
ModFile_in  = r'/home/vrath/work/MT/Annecy/ImageProc/In/ANN20_02_PT_NLCG_016'
ModFile_out = r'/home/vrath/work/MT/Annecy/ImageProc/Out/ANN20_02_PT_NLCG_016_insert'

In [ ]:
geocenter = [45.938251,     6.084900]
utm_x, utm_y = proj_latlon_to_utm(geocenter[0],geocenter[1],utm_zone =32631)
utmcenter =  [utm_x,utm_y, 0.]

In [ ]:
bodies = [['ellipsoid', 'replace',0.,    0., 0., 3000., 1000., 2000., 1000.,  0., 0., 30.],
          ['box',       'replace',0.,    0., 0., 1000., 2000., 1000., 1000.,  0., 0., 30.]]

In [ ]:
nb     = np.shape(bodies)

In [ ]:
# smoother=['gaussian',0.5]
smoother=['uniform',3]
total = 0
start = time.time()

In [ ]:
dx, dy, dz, rho, reference = readMod(ModFile_in+'.rho',out = True)
# writeMod(ModFile_out+'.rho', dx, dy, dz, rho,reference,out = True)
elapsed = (time.time() - start)
total = total + elapsed
print (' Used %7.4f s for reading model from %s ' % (elapsed,ModFile_in+'.rho'))

In [ ]:
air = rho > rhoair/100.

In [ ]:
rho = prepare_mod(rho,rhoair=rhoair)

In [ ]:
for ibody in range(nb[0]):
    body = bodies[ibody]
    rhonew = insert_body(dx,dy,dz,rho,body,smooth=smoother)
    rhonew[air] = rhoair
    Modout =ModFile_out+'_'+body[0]+str(ibody)+'_'+smoother[0]+'.rho'
    writeMod(Modout, dx, dy, dz, rhonew,reference,out = True)

    elapsed = (time.time() - start)
    print (' Used %7.4f s for processing/writing model to %s ' % (elapsed,Modout))
    print('\n')

In [ ]:
total = total + elapsed
print (' Total time used:  %f s ' % (total))